# Interface Options

In [1]:
#Teams Continuing to Reach the Playoffs or Consecutive Championships
#Interface Options
Playoff_Streak = 5
Champ_Streak = 1

---

# Import Libraries and Set Working Directory

In [2]:
#Import Libraries
import pandas as pd
import os
import math

In [3]:
#Working Directory
print(os.getcwd())

C:\Users\clinn\Documents\NBA\NBATeamRewards


In [4]:
#Change Working Directory
#os.chdir('C:\\Users\\clinn\\Documents')

In [5]:
#Read Data File
data = pd.read_excel('NBA_Playoff_Teams.xlsx')

#Or Read File Directly
#pd.read_excel('C:\\Users\\clinn\\Documents\\NBA_Playoff_Teams.xlsx')

---

# Data Overview

In [6]:
#Number of Observations
#len(data)

In [7]:
#Number of Columns
#len(data.columns)

In [8]:
#Column Names
#data.columns

In [9]:
#Firts 5 Rows
#data.head(5)

In [10]:
#Last 5 Rows
#data.tail(5)

In [11]:
#Column Info
#data.info()

In [12]:
#Column Stats
#data.describe().transpose()

---

# Consecutive Playoff Appearances

In [13]:
#Sort Data
data= data.sort_values(['Team', 'Year'], ascending=[1, 1])

In [14]:
#Create Variables - These variables will assist in identifing streaks for a particular team
#Dummy Year will be used to help identify when a streak began
data_PS = data
data_PS['Record_ID'] = range(1, 1+len(data_PS))
data_PS['Dummy_ID'] = data_PS.Record_ID + Playoff_Streak-1 
data_PS['Dummy_Year'] = data_PS.Year + Playoff_Streak-1 
#data_PS

In [15]:
#Join Data to itself to find records where year and ID match dummy variables. These will be teams that have a streak
data_PS = pd.merge(data_PS[['Year','Team','Champion','Record_ID']],
            data_PS[['Dummy_ID','Dummy_Year','Team']],
            left_on=['Record_ID','Team','Year'],
            right_on=['Dummy_ID','Team','Dummy_Year'],
            how='inner')
#data_PS

In [16]:
#Convert Dummy Variables to Start ID and Streak Begin
data_PS.Dummy_ID = data_PS.Record_ID - Playoff_Streak +1
data_PS.Dummy_Year = data_PS.Year - Playoff_Streak +1
data_PS.columns = ['Year', 'Team','Champion','Record_ID','Start_ID','Streak_Begin']
data_PS = data_PS[['Team', 'Year', 'Streak_Begin','Champion','Record_ID','Start_ID']]
#data_PS

In [17]:
#Identify when the next streak will begin after the current streak ends
data_PS['Nxt_Streak'] = data_PS.Streak_Begin + Playoff_Streak
#data_PS

In [18]:
#Join Data to itself to identify observations where the next streak (Nxt_Streak_y) can join back into the data
data_PS = pd.merge(data_PS[['Team','Year','Streak_Begin','Champion','Record_ID','Start_ID','Nxt_Streak']],
            data_PS[['Team','Nxt_Streak']],
            left_on=['Team','Streak_Begin'],
            right_on=['Team','Nxt_Streak'],
            how='left')
#data_PS

In [19]:
#Create new Dummy ID and join data to itself by team and id/dummy id
#The purpose of this is to see if the next row occurs within the streak of the previous row. 
#We will need to remove these in a following filter
data_PS['Dummy_ID'] = data_PS.Start_ID +1
data_PS = pd.merge(data_PS[['Team','Year','Streak_Begin','Champion','Record_ID','Start_ID','Nxt_Streak_x','Nxt_Streak_y']],
            data_PS[['Team','Streak_Begin','Dummy_ID']],
            left_on=['Team','Start_ID'],
            right_on=['Team','Dummy_ID'],
            how='left')
#data_PS

In [20]:
#Create new value for Streak Begin_y and filter data for rows that have a new streak or no consecutive rows in current streak
data_PS['Streak_Begin_y'] = data_PS.Streak_Begin_y + Playoff_Streak
data_PS_Streak_Start = data_PS[(data_PS.Streak_Begin_y != data_PS.Year)]
data_PS_Streak_Continue = data_PS = data_PS[pd.notnull(data_PS.Nxt_Streak_y)]
#data_PS = data_PS[pd.notnull(data_PS.Nxt_Streak_y) | (data_PS.Streak_Begin_y != data_PS.Year)]
#data_PS
#data_PS_Streak_Start
#data_PS_Streak_Continue

In [21]:
#Create a loop to go through iterations of joins to find playoff streaks
#Create a Dummy_Date to Join back to data. This will give a NAN for the first value
#This NAN is the beginning of a new streak and will be unioned to the Streak Start Data

while (len(data_PS_Streak_Continue) >0):
    data_PS_Streak_Continue['Dummy_Date'] = data_PS_Streak_Continue.Streak_Begin_y + 1
    data_PS_Streak_Continue = pd.merge(data_PS_Streak_Continue[['Team','Year','Streak_Begin_x','Champion','Record_ID','Start_ID','Nxt_Streak_x','Nxt_Streak_y','Streak_Begin_y','Dummy_ID']],
                data_PS_Streak_Continue[['Team','Dummy_Date']],
                left_on=['Team','Year'],
                right_on=['Team','Dummy_Date'],
                how='left') 
    data_PS_New_Streak = data_PS_Streak_Continue[pd.isnull(data_PS_Streak_Continue.Dummy_Date)]
    data_PS_New_Streak = data_PS_New_Streak.drop(columns=['Dummy_Date'])

    #Union dataframes together
    frames = [data_PS_Streak_Start, data_PS_New_Streak]
    data_PS_Streak_Start = pd.concat(frames, sort=False)

    #Remove the consecutuve years that fall between streaks
    data_PS_Streak_Continue['Nxt_Streak_y'] = data_PS_Streak_Continue.Nxt_Streak_y + Playoff_Streak
    data_PS_Streak_Continue = pd.merge(data_PS_Streak_Continue[['Team','Year','Streak_Begin_x','Champion','Record_ID','Start_ID','Nxt_Streak_x','Nxt_Streak_y','Streak_Begin_y','Dummy_ID']],
                data_PS_Streak_Continue[['Team','Nxt_Streak_y']],
                left_on=['Team','Streak_Begin_x'],
                right_on=['Team','Nxt_Streak_y'],
                how='left') 
    data_PS_Streak_Continue = data_PS_Streak_Continue[pd.notnull(data_PS_Streak_Continue.Nxt_Streak_y_y)]
    data_PS_Streak_Continue = data_PS_Streak_Continue.drop(columns=['Nxt_Streak_y_y'])
    data_PS_Streak_Continue.columns = ['Team','Year','Streak_Begin_x','Champion','Record_ID','Start_ID','Nxt_Streak_x','Nxt_Streak_y','Streak_Begin_y','Dummy_ID']
    data_PS_Streak_Continue['Nxt_Streak_y'] = data_PS_Streak_Continue.Nxt_Streak_y - Playoff_Streak   
    #data_PS_Streak_Continue
    #data_PS_Streak_Start
print ('Finished Playoff Streak Loop')

Finished Playoff Streak Loop


In [22]:
#Create Playoff Streaks Table - This identifies all of the streaks given the input parameters
Playoff_Streaks = data_PS_Streak_Start.sort_values(['Team', 'Year'], ascending=[1, 1])
Playoff_Streaks = Playoff_Streaks.reset_index()
Playoff_Streaks = Playoff_Streaks.drop(['Champion','Record_ID','Start_ID','Nxt_Streak_x','Nxt_Streak_y','Streak_Begin_y','Dummy_ID'], axis=1)
Playoff_Streaks = Playoff_Streaks[['Team','Streak_Begin_x','Year']]
Playoff_Streaks.columns = ['Team','Streak_Begin','Streak_End']
Playoff_Streaks

,Team,Streak_Begin,Streak_End
0,Atlanta Hawks,2008,2012
1,Atlanta Hawks,2013,2017
2,Boston Celtics,2008,2012
3,Boston Celtics,2015,2019
4,Chicago Bulls,2009,2013
5,Cleveland Cavaliers,2006,2010
6,Dallas Mavericks,2001,2005
7,Dallas Mavericks,2006,2010
8,Denver Nuggets,2004,2008
9,Denver Nuggets,2009,2013


In [23]:
#Create Playoff Streaks Reward Table - This identifies the total potential rewards for each team
Playoff_Streak_Rewards = Playoff_Streaks.groupby(['Team']).agg({'Streak_Begin':'count'})
Playoff_Streak_Rewards = Playoff_Streak_Rewards.reset_index()
Playoff_Streak_Rewards.columns = ['Team','Rewards_Count']
Playoff_Streak_Rewards

,Team,Rewards_Count
0,Atlanta Hawks,2
1,Boston Celtics,2
2,Chicago Bulls,1
3,Cleveland Cavaliers,1
4,Dallas Mavericks,2
5,Denver Nuggets,2
6,Detroit Pistons,1
7,Golden State Warriors,1
8,Houston Rockets,1
9,Indiana Pacers,1


In [24]:
#Calculate Total Rewards - This is a sum of all rewards
Total_Rewards = Playoff_Streak_Rewards.agg({'Rewards_Count':'sum'})
Total_Rewards = Total_Rewards.get(key = 'Rewards_Count')
print('Total Rewards:',Total_Rewards)

Total Rewards: 30


In [25]:
#Caluclate Reward Opportunity - This is the max amount of rewards that are possible for a team
analysisbegin = data.agg({'Year':'min'})
analysisbegin = analysisbegin.get(key = 'Year')
analysisend = data.agg({'Year':'max'})
analysisend = analysisend.get(key = 'Year')
analysisyears = analysisend - analysisbegin +1
Reward_Opportunity = analysisyears/Playoff_Streak
Reward_Opportunity = math.floor(Reward_Opportunity)
print('Reward Opportunity per Team:',Reward_Opportunity)

Reward Opportunity per Team: 4


In [26]:
#Calculate Total Potential Rewards - This is the max amount of rewards that are possible across the NBA
Total_Potential_Rewards = Reward_Opportunity*16
print('Total Potential Rewards:',Total_Potential_Rewards)

Total Potential Rewards: 64


In [27]:
#Calculate Rewards Achieve Percent - This is the % of rewards that were actually achieved vs what was possible
Rewards_Achieve_Pct = Total_Rewards/Total_Potential_Rewards
print('Percent of Rewards Achieved:', round((Rewards_Achieve_Pct*100),1),'%')

Percent of Rewards Achieved: 46.9 %


In [28]:
#Calculate Rewards per Year - This is the amount of rewards in a given year
Rewards_per_Year = Total_Rewards/analysisyears
print('Rewards per Year:',Rewards_per_Year)

Rewards per Year: 1.5
